<a href="https://colab.research.google.com/github/chi468/aiTemperature/blob/master/aiTemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

  # 커널 행렬에 L1 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
  #tf.keras.layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l1(0.01)),

  # 절편 벡터에 L2 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
  #tf.keras.layers.Dense(16, bias_regularizer=tf.keras.regularizers.l2(0.01)),

  # 시그모이드 활성화 층을 만듭니다:
  #tr.keras.layers.Dense(1, activation='sigmoid'),

  #simple_lstm_model.compile(optimizer='adam', loss='mse')


In [0]:
"""
dataX = sequence
dataY = target
data_dim = 40

train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX = np.array(dataX[0:train_size]) ,
testX = np.array(dataX[train_size:len(dataX)])
trainY = np.array(dataY[0:train_size])
testY = np.array(dataY[train_size:len(dataY)])

X = tf.Variable(tf.zeros(shape = (None,seq_length,data_dim)),dtype=float32)
Y = tf.Variable(tf.zeros(shape=(None,1)),dtype=float32)
"""

'\ndataX = sequence\ndataY = target\ndata_dim = 40\n\ntrain_size = int(len(dataY) * 0.7)\ntest_size = len(dataY) - train_size\ntrainX = np.array(dataX[0:train_size]) ,\ntestX = np.array(dataX[train_size:len(dataX)])\ntrainY = np.array(dataY[0:train_size])\ntestY = np.array(dataY[train_size:len(dataY)])\n\nX = tf.Variable(tf.zeros(shape = (None,seq_length,data_dim)),dtype=float32)\nY = tf.Variable(tf.zeros(shape=(None,1)),dtype=float32)\n'

In [0]:
"""
acc = tf.keras.metrics.Accuracy(
    name='accuracy', dtype=None
)

y00_pred = y00_pred.flatten()
testY = np.array(testY)
print(y00_pred)
print(testY)

plt.figure(figsize=(15,10))
plt.plot(y00_pred)
plt.plot(testY)
plt.legend(['pred','test'])

print(cal_res(y00_pred,testY))
"""

"\nacc = tf.keras.metrics.Accuracy(\n    name='accuracy', dtype=None\n)\n\ny00_pred = y00_pred.flatten()\ntestY = np.array(testY)\nprint(y00_pred)\nprint(testY)\n\nplt.figure(figsize=(15,10))\nplt.plot(y00_pred)\nplt.plot(testY)\nplt.legend(['pred','test'])\n\nprint(cal_res(y00_pred,testY))\n"

In [0]:
#!pip install tensorflow==2.0.0-rc1
#!pip install -q tensorflow-gpu==2.0.0-rc1

In [0]:
#print(tf.__version__)
#print(tf.test.is_gpu_available())

In [0]:
#%matplotlib inline
#import seaborn as sns #시각화 2
#from IPython.core.display import display, HTML   #노트북 화면이 작은 당신을 위한 코드
#display(HTML(">"))

In [0]:
 #DIR_DATA = "/content/gdrive/My Drive/aiTemp/"
 #DIR_TRAIN = DIR_DATA + "train.csv"
 #DIR_TEST = DIR_DATA + "test.csv"
 #DIR_SUB = DIR_DATA + "sample_submission.csv"

# train = pd.read_csv(DIR_TRAIN)
# test = pd.read_csv(DIR_TEST)
# sub = pd.read_csv(DIR_SUB)

In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [0]:
  from google.colab import files
  #files.upload()

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
temperature_name = ["X00","X07","X28","X31","X32"] #기온
localpress_name  = ["X01","X06","X22","X27","X29"] #현지기압
speed_name       = ["X02","X03","X18","X24","X26"] #풍속
water_name       = ["X04","X10","X21","X36","X39"] #일일 누적강수량
press_name       = ["X05","X08","X09","X23","X33"] #해면기압
sun_name         = ["X11","X14","X16","X19","X34"] #일일 누적일사량
humidity_name    = ["X12","X20","X30","X37","X38"] #습도
direction_name   = ["X13","X15","X17","X25","X35"] #풍향

In [0]:
# 기상청 데이터만 추출

#X14,X16,X19 제외

#X_train = train.loc[:,'X00':'X39']
#X_train = X_train.drop(['X14','X16','X19','X39'],axis=1)

req_x=['X00','X27','X24','X39','X33','X34','X12','X25']
X_train = train.loc[:,req_x]
test = test.loc[:,req_x]
y_columns = ['Y16']

X_trainLen = X_train.shape[1]

# standardization을 위해 평균과 표준편차 구하기
MEAN = X_train.mean()
STD = X_train.std()

# 표준편차가 0일 경우 대비하여 1e-07 추가
X_train = (X_train - MEAN) / (STD + 1e-07)

In [0]:
# RNN 모델에 입력 할 수 있는 시계열 형태로 데이터 변환 
def convert_to_timeseries(df, interval):
    sequence_list = []
    target_list = []
    
    for i in tqdm(range(df.shape[0] - interval)):
        sequence_list.append(np.array(df.iloc[i:i+interval,:-1]))
        target_list.append(df.iloc[i+interval,-1])
    
    sequence = np.array(sequence_list)
    target = np.array(target_list)
    
    return sequence, target

In [0]:
# t시점 이전 120분의 데이터로 t시점의 온도를 추정할 수 있는 학습데이터 형성
interval = 12;
#sequence = []
#for _ in range(17):
  #sequence.append(np.empty((0, interval, X_train.shape[1])))


# t시점 이전 120분의 데이터로 t시점의 온도를 추정할 수 있는 학습데이터 형성
sequence = np.empty((0, interval, X_train.shape[1]))
target = np.empty((0,))
for column in y_columns :
    
    concat = pd.concat([X_train, train[column]], axis = 1)

    _sequence, _target = convert_to_timeseries(concat.head(144*30), interval = 12)

    sequence = np.vstack((sequence, _sequence))
    target = np.hstack((target, _target))


100%|██████████| 4308/4308 [00:01<00:00, 3118.45it/s]


In [0]:
from keras import backend as K

#손실함수 4제곱으로 직접 정의
def msse(y_pred, y_true):
    loss = K.mean((y_true-y_pred)**4)
 
    return loss

Using TensorFlow backend.


In [0]:
# 간단한 lstm 모델 구축하기
simple_lstm_model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(128, input_shape=sequence.shape[-2:]),

  tf.keras.layers.Dense(128, activation='linear'),
  
  tf.keras.layers.Dense(1)

])


simple_lstm_model.compile(optimizer='adam', loss='mse')

In [0]:
# loss가 4미만으로 떨어지면 학습 종료 시키는 기능
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if(logs.get('loss') < 0.25):
            print('\n Loss is under 0.25, cancelling training')
            self.model.stop_training = True

In [0]:
callbacks = myCallback()

In [0]:
"""
dataX = sequence
dataY = target
data_dim = X_trainLen

train_size = int(len(dataY))
output_size = 3 * 144
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])
"""

dataX = sequence
dataY = target
data_dim = X_trainLen

train_size = int(len(dataY)*0.8)
test_size = len(dataY) - train_size
trainX = np.array(dataX[0:train_size]) ,
testX = np.array(dataX[train_size:len(dataX)])
trainY = np.array(dataY[0:train_size])
testY = np.array(dataY[train_size:len(dataY)])


In [0]:
# 모델 학습
simple_lstm_model.fit(    
    sequence, target,
    epochs=100,
    batch_size=128,
    verbose=0,
    shuffle=False,
    callbacks = [callbacks]
)

In [0]:
def cal_res(pred,test):
  res = 0.
  for i in range(len(pred)):
     res += abs((pred[i]-test[i]))
  return res / len(pred)

In [0]:
testX, _ = convert_to_timeseries(pd.concat([X_train.tail(144*3+12),train['Y00'].tail(144*3+12)],axis=1), interval = 12)
y_pred = pd.DataFrame(simple_lstm_model.predict(testX))
cor_test = pd.concat([y_pred, train['Y18'].tail(144*3).reset_index(drop=True)],ignore_index=True,axis=1).corr()
print(cor_test)
#cor = cor_test.corr()
#print(cor)

#testY = np.array(testY)
'''
print(y_pred)
print(testY)


plt.figure(figsize=(15,10))
plt.plot(y_pred)
plt.plot(testY)
plt.legend(['pred','test'])

print(cal_res(y_pred,testY))
'''

100%|██████████| 432/432 [00:00<00:00, 2301.21it/s]


          0         1
0  1.000000  0.915008
1  0.915008  1.000000


"\nprint(y_pred)\nprint(testY)\n\n\nplt.figure(figsize=(15,10))\nplt.plot(y_pred)\nplt.plot(testY)\nplt.legend(['pred','test'])\n\nprint(cal_res(y_pred,testY))\n"

In [0]:
testY = train['Y16']
plt.figure(figsize=(15,10))
plt.plot(testY)
plt.legend(['test'])

In [0]:


# convert_to_timeseries 함수를 쓰기 위한 dummy feature 생성
X_train['dummy'] = 0
# train set에서 도출된 평균과 표준편차로 standardization 실시 
test = (test - MEAN) / (STD + 1e-07)
# convert_to_timeseries 함수를 쓰기 위한 dummy feature 생성
test['dummy'] = 0
# train과 test 기간을 합쳐서 120분 간격으로 학습데이터 재구축
X_test, _ = convert_to_timeseries(pd.concat([X_train, test], axis = 0), interval=12)
# test set 기간인 후반부 80일에 맞게 자르기 
X_test = X_test[-11520:, :, :]
# 만들어 두었던 dummy feature 제거
X_train.drop('dummy', axis = 1, inplace = True)
test.drop('dummy', axis = 1, inplace = True)

print(test.shape)

100%|██████████| 16260/16260 [00:10<00:00, 1570.05it/s]


(11520, 8)


In [0]:
# LSTM 레이어는 고정
simple_lstm_model.layers[0].trainable = False
# fine tuning 할 때 사용할 학습데이터 생성 (Y18)
finetune_X, finetune_y = convert_to_timeseries(pd.concat([X_train.tail(432), train['Y18'].tail(432)], axis = 1), interval=12)

# LSTM 레이어는 고정 시켜두고, DNN 레이어에 대해서 fine tuning 진행 (Transfer Learning)
finetune_history = simple_lstm_model.fit(
            finetune_X, finetune_y,
            epochs=200,
            batch_size=64,
            shuffle=False,
            verbose = 0)



100%|██████████| 420/420 [00:00<00:00, 1878.25it/s]


In [0]:
# 예측하기 
print(X_test.shape)
finetune_pred = simple_lstm_model.predict(X_test)

# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':finetune_pred.reshape(1,-1)[0]})

#print(submit)


csvName = "result"

a = 0
"""
while(True):
  try :
    submit.to_csv(csvName+str(a)+'.csv', index = False)
    files.download(csvName+str(a)+'.csv')
    break
  except MessageError:
    a = a + 1
"""


(11520, 12, 8)


"\nwhile(True):\n  try :\n    submit.to_csv(csvName+str(a)+'.csv', index = False)\n    files.download(csvName+str(a)+'.csv')\n    break\n  except MessageError:\n    a = a + 1\n"

In [0]:

acc = tf.keras.metrics.Accuracy(
    name='accuracy', dtype=None
)
testY = np.array(testY)

plt.figure(figsize=(15,10))
plt.plot(testY)
plt.legend(['test'])